### This notebook is ran with GPU and for simplicity and reproducibility reasons code is ajusted for Colab environment

In [1]:
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score, make_scorer, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

from tabpfn import TabPFNClassifier
import torch

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = '/content/drive/MyDrive/{your_folder_path}/'

df = pd.read_csv(f'{DATA_DIR}/model_ready_dataset.csv')

print(df.columns)
df.shape

Index(['book_pages', 'customer_library_distance', 'book_price', 'book_title',
       'book_authors_wendy nelson espeland',
       'book_authors_oscar adolph leutwiler',
       'book_authors_axel petrus johnson', 'book_authors_william stokes',
       'book_categories_libraries', 'book_authors_kim starkey jonker',
       'book_authors_murthy', 'book_authors_les carlson',
       'book_authors_samuel milroy ballard', 'book_authors_samuel newth',
       'book_categories_water supply',
       'book_categories_language arts & disciplines',
       'book_authors_stanisław lem', 'book_authors_richard schmalensee',
       'book_authors_bruce m. becker', 'book_authors_michael sauder',
       'book_authors_gary copeland', 'book_categories_geology', 'book_age',
       'book_authors_gordon w. stewart',
       'book_authors_sir william james moore', 'is_late'],
      dtype='object')


(1544, 26)

In [5]:
df['is_late'].value_counts()

,count
is_late,
0,1383
1,161


In [6]:
X = df.drop(columns=['is_late'])
y = df['is_late']

In [7]:
n_iter=25
n_splits=5
seed=25

In [8]:
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)


X_scaled = StandardScaler().fit_transform(X)
tabpfn_model = TabPFNClassifier(device='cuda' if torch.cuda.is_available() else 'cpu')
tabpfn_model.fit(X_scaled, y)

y_pred = cross_val_predict(tabpfn_model, X_scaled, y, cv=skf, method='predict')
y_proba_all = cross_val_predict(tabpfn_model, X_scaled, y, cv=skf, method='predict_proba')

if y_proba_all.shape[1] == 2:
    y_proba = y_proba_all[:, 1]
else:
    y_proba = np.max(y_proba_all, axis=1)

f1 = f1_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
pr_auc = average_precision_score(y, y_proba)

print(f1, pr_auc, recall, precision)

/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:89: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-classifier.ckpt.
  model, _, config_ = load_model_criterion_config(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tabpfn-v2-classifier.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

0.24644549763033174 0.301912420080635 0.16149068322981366 0.52


#### Results for F1, Precision, Recall, PR AUC:
####    0.2464 0.3019 0.1615 0.52